In [14]:
#%%
import sys
import pandas as pd
import psycopg2
from pathlib import Path

from dateutil.parser import parse

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

from connection import TRACK_DB_CONN, SCORE_DB_STR

pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

In [2]:
drive_client = GoogleDriveClient( )
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient( )
ggsheet_repo = GoogleSheetsService(ggsheet_client)

sa_ana = Path('/home/linhnguyen/.config/ts-dp-v2.json')
adc = Path('/home/linhnguyen/.config/gcloud/application_default_credentials.json')
bq_client = BigqueryClient(billing_project_id='prj-ts-p-analytic-8057',  sa_file_path = sa_ana) #adc_file_path= adc)
bq_repo = BigqueryService(bq_client)

1. LIST LEAD ID NON EXIST

In [19]:
list_lead = ['150209','83162', '134627', '92042', '115691', '110561']
rq_vnpt = pd.read_sql("""select id as telco_lead_id,
                        created_at,
                        JSON_VALUE(downstream_param,'$.lead_id') as vnpt_lead_id
                  from telco_gateway_vinaphone.general_requests
                  where client_code = 'tpbank' and request_type = 'forward_lead' 
                      """, con=TRACK_DB_CONN)

# # Print the first few rows of the data
rq_vnpt = rq_vnpt[rq_vnpt['vnpt_lead_id'].isin(list_lead)]
rq_vnpt['vnpt_lead_id'] = rq_vnpt['vnpt_lead_id'].astype('int64')
rq_vnpt

,telco_lead_id,created_at,vnpt_lead_id
1442,87105,2022-04-27 09:21:08,83162
3745,101704,2022-05-02 13:32:18,92042
7822,128414,2022-05-09 12:32:12,110561
8713,137910,2022-05-13 15:30:15,115691
10274,167591,2022-05-23 08:19:36,134627
11337,196281,2022-06-04 08:43:54,150209


In [ ]:
tpb_query = f"""
with tbl1 as (
  SELECT distinct 
  application_id,
  CAST(JSON_VALUE(result_data, '$.id') AS INTEGER) AS offer_qualifications_id
  FROM `prj-ts-p-lakehouse-8435.dop.application_services`
)
, tbl2 AS (
  SELECT DISTINCT 
  id as offer_qualifications_id
  , CAST(JSON_VALUE(result, '$.other.lead_id') AS INTEGER) AS telco_lead_id
  FROM `prj-ts-p-lakehouse-8435.v1_proxy.offer_qualifications` 
)

, mapping AS (
  SELECT 
  tbl1.application_id
  , tbl2.telco_lead_id
  FROM tbl1 JOIN tbl2 USING(offer_qualifications_id)
)

  select smv.* , mapping.telco_lead_id
  FROM `shared_dop.tpbank_daily_data_approved_and_active_for_recon` smv
  LEFT JOIN mapping USING (application_id)
  WHERE (is_actived=1
  AND card_active_date >= date_trunc(date_sub(current_date, interval 4 month), month)  
  AND card_active_date < date_trunc(current_date, month)) or 
  (uw_approved_date >= date_trunc(date_sub(current_date, interval 4 month), month)  
  AND uw_approved_date < date_trunc(date_sub(current_date, interval 3 month), month));"""

tpb_df = bq_repo.get_data_from_query_into_pandas(query_str= tpb_query)
tpb_df

In [20]:
phone_infos = pd.read_sql("""
SELECT 
    telco_lead_id, 
    phone_number, 
    bank_code,
    bound_code, 
    score, 
    score_range,
    to_char(sent_at AT TIME ZONE 'UTC','YYYY-MM-DD') as sent_date,
    to_char(sent_at AT TIME ZONE 'UTC','YYYY-MM') as batch_month,
    (case when (other ->> 'sender')::text is null and (other ->> 'channel')::text is null then 'sms' 
        when (other ->> 'sender')::text = 'sms' then 'sms'
        when (other ->> 'sender')::text = 'avay' then 'avay' 
        when (other ->> 'sender')::text = 'viet_tin' then 'viet_tin' 
        end) as channel,
    telco_code,
    was_sent
FROM phone_infos 
WHERE bank_code = 'tpbank' and telco_code = 'vinaphone'
    
""", con = SCORE_DB_STR)

In [18]:
phone_infos[phone_infos['telco_lead_id'].isin(rq_vnpt['telco_lead_id'])].merge(rq_vnpt, how = 'left', on = 'telco_lead_id')

,telco_lead_id,phone_number,bank_code,bound_code,score,score_range,sent_date,batch_month,channel,telco_code,was_sent,created_at,vnpt_lead_id
0,87105,84946651285,tpbank,TPB_01,None,685-689,2022-04-27,2022-04,None,vinaphone,True,2022-04-27 09:21:08,83162
1,101704,841653883408,tpbank,TPB_01,None,730-734,2022-05-02,2022-05,None,vinaphone,True,2022-05-02 13:32:18,92042
2,128414,84918345365,tpbank,TPB_01,None,775-779,2022-05-09,2022-05,None,vinaphone,True,2022-05-09 12:32:12,110561
3,137910,841235804804,tpbank,TPB_01,None,655-659,2022-05-13,2022-05,None,vinaphone,True,2022-05-13 15:30:15,115691
4,167591,84886999285,tpbank,TPB_01,None,845-849,2022-05-23,2022-05,None,vinaphone,True,2022-05-23 08:19:36,134627
5,196281,84947871114,tpbank,TPB_01,None,725-729,2022-06-04,2022-06,None,vinaphone,True,2022-06-04 08:43:54,150209


In [22]:
phone_infos[phone_infos['telco_lead_id'] == 101704]

,telco_lead_id,phone_number,bank_code,bound_code,score,score_range,sent_date,batch_month,channel,telco_code,was_sent
16663,101704,841653883408,tpbank,TPB_01,None,730-734,2022-05-02,2022-05,None,vinaphone,True
